# Approach: Topological Sort Using Kahn's Algorithm

### Intuition

To solve the problem, we must first consider when a node is safe or unsafe. If we begin at any node and proceed along any path from that node, we will eventually reach either a terminal node or enter a cycle and continue to loop in it without ever reaching a terminal node.

If there is no path from the node that enters a cycle, we will always be able to reach a terminal node. As a result, such a node is a safe node and should be added to our answer array.

The problem is reduced to finding the nodes that do not have any paths that lead to a cycle.

Intuitively, we can realize that a node is safe if all of its outgoing edges are to nodes that are also safe. This is due to the fact that if no neighbor leads to a cycle, no path from the node can either.

We know the terminal nodes are safe. As a result, nodes that solely have outgoing edges to terminal nodes are eventually safe nodes. Then we may check the nodes that have just outgoing edges to safe nodes again and keep updating until no new safe node is discovered.

The question is, how do we efficiently traverse from terminal nodes to nodes that only have outgoing edges to terminal nodes? We can reverse the edges of the graph to create a new graph with reversed edges. After we have visited all of the terminal nodes, we can use this new graph to go to the nodes that have edges to the terminal nodes in the original graph by using the reverse edges that we added.

Let's put this new graph to use now. A node is a safe node if all of its incoming edges come from previously identified safe nodes in the graph. If we erase the edges outgoing from the safe node and discover a node with no incoming edges, it is a new safe node. This gives us hints for thinking about Kahn's method, which does a topological sort by removing the edges in the exact way we want.

A topological sort or topological ordering of a directed graph is a linear ordering of its vertices such that for every directed edge u -> v from vertex u to vertex v, u comes before v in the ordering.

In a directed acyclic graph, we can use Kahn's algorithm to get the topological ordering. Kahn’s algorithm works by keeping track of the number of incoming edges into each node (indegree). It works by repeatedly visiting the nodes with an indegree of zero and deleting all the edges associated with it leading to a decrement of indegree for the nodes whose incoming edges are deleted. This process continues until no elements with zero indegree can be found.

If you are not familiar with Kahn's algorithm, we suggest you read our LeetCode Explore Card.

The advantage of using Kahn's technique is that it also aids in the discovery of graph cycles. The Kahn's method does not visit any node in a cycle. As a result, nodes with outgoing edges from nodes in the cycle (in this reversed graph) will never be visited and so will never be marked safe. Nodes with outgoing edges from these unsafe nodes will never be visited as well, and so on. Basically, every node in the original network that has a path to the cycle will never be visited by Kahn's algorithm, which is exactly what we want.

We can see that if there is a cycle, the indegree of nodes in the cycle cannot be set to 0 due to cyclic dependency. We are unable to visit the cycle's nodes. We are also unable to visit any node with an incoming edge from any node in the cycle. Similarly, realize that any node with an incoming edge from nodes 3 or 5 would not have been visited as well.

### Algorithm

Create an integer n equal to the length of graph to get the number of nodes in the given graph.
Create an array indegree of length n where indegree[x] stores the number of edges entering node x.
We create an adjacency list adj in which adj[x] contains all the nodes with an incoming edge from node x, i.e., neighbors of node x. We create this adjacency list by iterating over graph and adding the reverse edges. For a node i which originally has outgoing edges to nodes in graph[i], we push i into adj[node] to add a reverse edge from node to i.
Initialize a queue of integers q and start a BFS algorithm moving from the leaf nodes to the parent nodes.
Begin the BFS traversal by pushing all of the leaf nodes (indegree equal to 0) in the queue.
Create a boolean array safe of size n to track the safe nodes in the graph.
While the queue is not empty;
Dequeue the first node from the queue.
Mark node as safe.
For each neighbor (nodes that have an incoming edge from node) of node, we decrement indegree[neighbor]by 1 to delete the node -> neighbor edge.
If indegree[neighbor] == 0, it means that neighbor behaves as a leaf node, so we push neighbor in the queue.
Create an answer array safeNodes of size n. Iterate over all the nodes from 0 to n - 1 and add all the safe nodes in safeNodes.
Return safeNodes.

In [3]:
from typing import List
from collections import deque

class Solution:
    def eventualSafeNodes(self, graph: List[List[int]]) -> List[int]:
        n = len(graph)
        indegree = [0] * n
        adj = [[] for _ in range(n)]

        for i in range(n):
            for node in graph[i]:
                adj[node].append(i)
                indegree[i] += 1

        q = deque()
        # Push all the nodes with indegree zero in the queue.
        for i in range(n):
            if indegree[i] == 0:
                q.append(i)

        safe = [False] * n
        while q:
            node = q.popleft()
            safe[node] = True

            for neighbor in adj[node]:
                # Delete the edge "node -> neighbor".
                indegree[neighbor] -= 1
                if indegree[neighbor] == 0:
                    q.append(neighbor)

        safeNodes = []
        for i in range(n):
            if safe[i]:
                safeNodes.append(i)

        return safeNodes

In [4]:
graph_1 = [[1,2],[2,3],[5],[0],[5],[],[]]
print(Solution().eventualSafeNodes(graph_1))

graph_2 = [[1,2],[2,3],[5],[0],[5],[2],[]]
print(Solution().eventualSafeNodes(graph_2))

graph_3 = [[1,2,3,4],[1,2],[3,4],[0,4],[]]
print(Solution().eventualSafeNodes(graph_3))

[2, 4, 5, 6]
[6]
[4]
